<a href="https://colab.research.google.com/github/KIRTAN28/industry4.0-predictive-maintenance-deep-rl/blob/main/RL_single_agent_code2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import gymnasium as gym
from gymnasium import spaces
import numpy as np
import math
from typing import Optional, Union, Tuple, Dict
from collections import deque
import random
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, InputLayer
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import MeanSquaredError

# --- ENVIRONMENT: PredictiveMaintenanceEnv ---

class PredictiveMaintenanceEnv(gym.Env):
    """
    Observation space (11 features per component):
    [Tᵢ, tᵢ, Δt_last, λᵢ, Rᵢ, RULᵢ, MTTFᵢ, Lᵢ, MTTRᵢ, a_prev, Cᵢ]

    Features are NORMALIZED to range [0, 1] for the agent.
    """
    metadata = {"render_modes": ["human"], "render_fps": 1}

    def __init__(self, time_step: float = 1.0, render_mode: Optional[str] = None):
        super(PredictiveMaintenanceEnv, self).__init__()

        # Environment Configuration
        self.TIME_STEP = time_step
        self.render_mode = render_mode

        # Termination parameters
        self.MAX_AGE_TERMINATION = 10000.0
        self.RELIABILITY_THRESHOLD = 0.15

        # Weibull parameters for components
        self.components = {
            "Electrodes" : {"beta" : 1.61,"eta" : 2388}
        }

        self.num_components = len(self.components)
        self.component_names = list(self.components.keys())

        # Initial Fixed Cost Parameters (Constants)
        self.COST_OP_HOUR_BASE_INITIAL = 2.0
        self.COST_DOWN_HOUR_INITIAL = 500.0

        # Cost Parameters
        self.OP_HAZARD_SCALE = 40000.0
        self.COST_PM_FIXED_BASE = 3000.0
        self.COST_CM_FIXED_BASE = 15000.0
        self.PENALTY_UNPLANNED_FAIL = 100000.0
        self.DOWNTIME_PM_HRS = 8.0
        self.MTTR_R_PENALTY = 5.0

        # Imperfect Maintenance Parameter
        self.PM_TARGET_RELIABILITY_R = 0.90

        # State Initialization
        self.weibull_ages = np.zeros(self.num_components)
        self.absolute_ages = np.zeros(self.num_components)

        self.time_since_maintenance = np.zeros(self.num_components)
        self.last_action = np.zeros(self.num_components)
        self.load_fraction = np.ones(self.num_components) * 0.8

        # Tracks the reliability ceiling
        self.max_achievable_reliability = np.ones(self.num_components) * 1.0

        # DYNAMIC COST TRACKING
        self.current_base_op_cost = np.ones(self.num_components) * self.COST_OP_HOUR_BASE_INITIAL
        self.current_base_down_cost = np.ones(self.num_components) * self.COST_DOWN_HOUR_INITIAL

        # Spaces
        self.action_space = spaces.MultiDiscrete([3] * self.num_components)

        # Observation space is normalized [0, 1]
        self.observation_space = spaces.Box(
            low=0.0, high=1.0, shape=(self.num_components, 11), dtype=np.float32,
        )

    # --- Weibull helper functions ---

    def _calculate_age_for_reliability(self, target_R: float, component_index: int) -> float:
        params = self.components[self.component_names[component_index]]
        beta, eta = params["beta"], params["eta"]
        try:
            target_R = max(1e-9, min(target_R, 1.0 - 1e-9))
            t_prime = eta * ((-math.log(target_R)) ** (1 / beta))
            return t_prime
        except ValueError:
            return 0.0

    def hazard_rate(self, t: float, beta: float, eta: float) -> float:
        if t <= 0.0: return 0.0
        return (beta / eta) * ((t / eta) ** (beta - 1))

    def reliability(self, t: float, beta: float, eta: float) -> float:
        return np.exp(-((t / eta) ** beta))

    def mean_time_to_failure(self, beta: float, eta: float) -> float:
        return eta * math.gamma(1 + 1 / beta)

    def remaining_useful_life(self, t: float, beta: float, eta: float, threshold: float = 0.1) -> float:
        try:
            R_current = self.reliability(t, beta, eta)
            if R_current <= threshold: return 0.0
            t_threshold = eta * ((-math.log(threshold)) ** (1 / beta))
            RUL = t_threshold - t
            return max(0.0, RUL)
        except ValueError:
            return 0.0

    def mean_time_to_repair(self, reliability: float) -> float:
        scaling_factor = 1.0 + (self.MTTR_R_PENALTY * (1.0 - reliability))
        CMT_rate = 0.05
        TTAS_rate = 0.1
        TTAP_rate = 0.1
        CMT = (1 / CMT_rate) * scaling_factor
        TTAS = (1 / TTAS_rate) * scaling_factor
        TTAP = (1 / TTAP_rate) * scaling_factor
        MDT = max(TTAS, TTAP)
        MTTR = CMT + MDT
        return MTTR

    # --- Environment core methods ---
    def reset(self, seed: Optional[int] = None, options: Optional[dict] = None) -> Tuple[np.ndarray, Dict]:
        super().reset(seed=seed)

        self.weibull_ages.fill(0.0)
        self.absolute_ages.fill(0.0)
        self.time_since_maintenance.fill(0.0)
        self.last_action.fill(0.0)

        self.max_achievable_reliability.fill(1.0)
        self.current_base_op_cost.fill(self.COST_OP_HOUR_BASE_INITIAL)
        self.current_base_down_cost.fill(self.COST_DOWN_HOUR_INITIAL)

        return self._get_state(), {}

    def step(self, actions: Union[np.ndarray, list]) -> Tuple[np.ndarray, float, bool, bool, Dict]:
        actions = np.array(actions)
        total_reward = 0.0
        info = {}

        terminate_max_age = False
        terminate_low_reliability = False
        running_cost_total = 0.0

        for i, (name, params) in enumerate(self.components.items()):
            beta, eta = params["beta"], params["eta"]

            effective_age = self.weibull_ages[i]

            hazard = self.hazard_rate(effective_age, beta, eta)
            reliability = self.reliability(effective_age, beta, eta)
            component_mttr = self.mean_time_to_repair(reliability)

            # 1. DYNAMIC OPERATION COST
            # CLAMPING COST: Prevent infinite cost explosion for network stability
            raw_op_cost = self.current_base_op_cost[i] * (1.0 + self.OP_HAZARD_SCALE * hazard)
            cost_op = min(raw_op_cost, 50000.0) # Cap cost at 50k to prevent NaNs

            running_cost_total += cost_op

            prob_failure = 1.0 - np.exp(-hazard * self.TIME_STEP)
            failed_spontaneously = np.random.rand() < prob_failure

            # --- Maintenance Handling ---
            maintenance_occurred = False

            if actions[i] == 1:  # PM
                maintenance_occurred = True
                cost_pm_fixed = self.COST_PM_FIXED_BASE
                cost_downtime = self.current_base_down_cost[i] * self.DOWNTIME_PM_HRS
                total_reward -= (cost_pm_fixed + cost_downtime)

                R_target_old_max = self.max_achievable_reliability[i]
                R_target_new_max = R_target_old_max * self.PM_TARGET_RELIABILITY_R

                t_prime = self._calculate_age_for_reliability(R_target_new_max, i)
                self.weibull_ages[i] = t_prime
                self.time_since_maintenance[i] = 0.0
                self.max_achievable_reliability[i] = R_target_new_max

                # print(f"  --- PM Executed. R_old={reliability:.4f} -> R_new_max={R_target_new_max:.4f} ---")

            elif actions[i] == 2:  # CM
                maintenance_occurred = True
                dynamic_cm_fixed_cost = self.COST_CM_FIXED_BASE
                cost_downtime = self.current_base_down_cost[i] * component_mttr
                cost_cm = dynamic_cm_fixed_cost

                total_reward -= (cost_cm + cost_downtime)

                self.current_base_op_cost[i] = self.COST_OP_HOUR_BASE_INITIAL
                self.current_base_down_cost[i] = self.COST_DOWN_HOUR_INITIAL

                self.weibull_ages[i] = 0.0
                self.time_since_maintenance[i] = 0.0
                self.max_achievable_reliability[i] = 1.0

            elif actions[i] == 0: # NOOP
                if failed_spontaneously:
                    maintenance_occurred = True
                    dynamic_cm_fixed_cost = self.COST_CM_FIXED_BASE
                    cost_downtime = self.current_base_down_cost[i] * component_mttr
                    cost_cm = dynamic_cm_fixed_cost + self.PENALTY_UNPLANNED_FAIL

                    total_reward -= (cost_cm + cost_downtime)

                    self.current_base_op_cost[i] = self.COST_OP_HOUR_BASE_INITIAL
                    self.current_base_down_cost[i] = self.COST_DOWN_HOUR_INITIAL

                    self.weibull_ages[i] = 0.0
                    self.time_since_maintenance[i] = 0.0
                    self.max_achievable_reliability[i] = 1.0

            # Age update
            self.absolute_ages[i] += self.TIME_STEP
            if not maintenance_occurred:
                self.weibull_ages[i] += self.TIME_STEP
                self.time_since_maintenance[i] += self.TIME_STEP

            # Termination conditions
            if self.weibull_ages[i] >= self.MAX_AGE_TERMINATION:
                terminate_max_age = True

            current_reliability = self.reliability(self.weibull_ages[i], beta, eta)
            if current_reliability <= self.RELIABILITY_THRESHOLD:
                terminate_low_reliability = True

            self.last_action[i] = actions[i]

        total_reward -= running_cost_total * self.TIME_STEP
        done = terminate_max_age or terminate_low_reliability
        next_state = self._get_state()

        return next_state, total_reward, done, False, info

    # EXPERT FIX: Normalize state for Neural Network stability
    def _get_state(self) -> np.ndarray:
        state = []
        for i, (name, params) in enumerate(self.components.items()):
            beta, eta = params["beta"], params["eta"]

            weibull_age = self.weibull_ages[i]
            absolute_age = self.absolute_ages[i]

            hazard = self.hazard_rate(weibull_age, beta, eta)
            reliability = self.reliability(weibull_age, beta, eta)
            RUL = self.remaining_useful_life(weibull_age, beta, eta)
            MTTF = self.mean_time_to_failure(beta, eta)
            MTTR = self.mean_time_to_repair(reliability)
            load_fraction = self.load_fraction[i]
            confidence = reliability

            # NORMALIZATION FACTORS
            MAX_TIME = self.MAX_AGE_TERMINATION
            MAX_MTTR = 2000.0 # Approx max MTTR

            s_i = [
                absolute_age / MAX_TIME,                   # 1. T_abs (Normalized)
                weibull_age / MAX_TIME,                    # 2. t_weibull (Normalized)
                self.time_since_maintenance[i] / MAX_TIME, # 3. TSM (Normalized)
                min(hazard, 1.0),                          # 4. Hazard (Clipped/Norm)
                reliability,                               # 5. R (Already 0-1)
                RUL / MAX_TIME,                            # 6. RUL (Normalized)
                MTTF / MAX_TIME,                           # 7. MTTF (Normalized)
                load_fraction,                             # 8. Load (Already 0-1)
                MTTR / MAX_MTTR,                           # 9. MTTR (Normalized)
                self.last_action[i] / 2.0,                 # 10. Action (0, 0.5, 1.0)
                confidence,                                # 11. C (Already 0-1)
            ]
            state.append(s_i)
        return np.array(state, dtype=np.float32)

    def render(self):
        pass

    def close(self):
        pass

# --- DDQN AGENT IMPLEMENTATION ---

class ReplayBuffer:
    def __init__(self, capacity: int):
        self.buffer = deque(maxlen=capacity)

    def push(self, state, action, reward, next_state, done):
        self.buffer.append((state, action, reward, next_state, done))

    def sample(self, batch_size: int):
        if len(self.buffer) < batch_size: return []
        return random.sample(self.buffer, batch_size)

    def __len__(self):
        return len(self.buffer)

class DDQNAgent:
    def __init__(self, state_shape, action_dim, gamma=0.99, lr=0.001, epsilon=1.0, epsilon_min=0.01, epsilon_decay=0.9999, buffer_size=50000, batch_size=64, update_target_freq=100):
        self.state_shape = state_shape
        self.action_dim = action_dim
        self.num_components = state_shape[0]
        self.output_dim = self.action_dim ** self.num_components
        self.gamma = gamma
        self.lr = lr
        self.epsilon = epsilon
        self.epsilon_min = epsilon_min
        self.epsilon_decay = epsilon_decay
        self.batch_size = batch_size
        self.update_target_freq = update_target_freq
        self.memory = ReplayBuffer(buffer_size)
        self.train_step_count = 0
        self.q_local = self._build_model()
        self.q_target = self._build_model()
        self.q_target.set_weights(self.q_local.get_weights())

    def _build_model(self):
        model = Sequential()
        model.add(InputLayer(input_shape=self.state_shape))
        model.add(tf.keras.layers.Flatten())

        # EXPERT FIX: He Initialization for ReLU
        model.add(Dense(256, activation='relu', kernel_initializer='he_normal'))
        model.add(Dense(128, activation='relu', kernel_initializer='he_normal'))
        model.add(Dense(self.output_dim, activation='linear')) # Linear is correct for Q-values

        # EXPERT FIX: Gradient Clipping
        model.compile(loss=MeanSquaredError(), optimizer=Adam(learning_rate=self.lr, clipnorm=1.0))
        return model

    def get_action(self, state: np.ndarray) -> np.ndarray:
        if np.random.rand() <= self.epsilon:
            actions = np.random.randint(0, self.action_dim, size=self.num_components)
        else:
            state = np.expand_dims(state, axis=0)
            q_values = self.q_local.predict(state, verbose=0)[0]
            flat_action = np.argmax(q_values)
            actions = self._flat_to_multi_discrete(flat_action)
        return actions

    def _flat_to_multi_discrete(self, flat_action: int) -> np.ndarray:
        actions = []
        base = self.action_dim
        temp_action = flat_action
        for _ in range(self.num_components):
            action_i = temp_action % base
            actions.append(action_i)
            temp_action //= base
        return np.array(actions[::-1])

    def _multi_discrete_to_flat(self, actions: np.ndarray) -> int:
        flat_action = 0
        base = self.action_dim
        for i, action_i in enumerate(reversed(actions)):
            flat_action += action_i * (base ** i)
        return flat_action

    def train(self):
        if len(self.memory) < self.batch_size:
            return
        batch = self.memory.sample(self.batch_size)
        states = np.array([exp[0] for exp in batch])
        actions_md = np.array([exp[1] for exp in batch])
        rewards = np.array([exp[2] for exp in batch])
        next_states = np.array([exp[3] for exp in batch])
        dones = np.array([exp[4] for exp in batch])

        q_values_local = self.q_local.predict(states, verbose=0)
        q_next_local = self.q_local.predict(next_states, verbose=0)
        max_action_indices_flat = np.argmax(q_next_local, axis=1)
        q_next_target = self.q_target.predict(next_states, verbose=0)
        q_target_evaluated = q_next_target[np.arange(self.batch_size), max_action_indices_flat]
        target_q_values = rewards + self.gamma * q_target_evaluated * (1 - dones)

        q_targets = q_values_local.copy()
        for i in range(self.batch_size):
            flat_action = self._multi_discrete_to_flat(actions_md[i])
            q_targets[i, flat_action] = target_q_values[i]

        self.q_local.fit(states, q_targets, epochs=1, verbose=0)
        self.train_step_count += 1
        if self.train_step_count % self.update_target_freq == 0:
            self.q_target.set_weights(self.q_local.get_weights())
        if self.epsilon > self.epsilon_min:
            self.epsilon *= self.epsilon_decay

# --- TRAINING EXECUTION ---

REWARD_SCALE_FACTOR = 1000.0

def train_ddqn():
    tf.random.set_seed(42)
    np.random.seed(42)
    random.seed(42)

    env = PredictiveMaintenanceEnv(time_step=1.0)
    state_shape = env.observation_space.shape
    action_dim = env.action_space.nvec[0]
    num_components = env.num_components

    agent = DDQNAgent(
        state_shape=state_shape,
        action_dim=action_dim,
        lr=0.0005,
        epsilon_decay=0.9995,
        buffer_size=50000,
        batch_size=32,
        update_target_freq=50
    )

    EPISODES = 100
    ACTION_MAP = {0: "NOOP (Do Nothing)", 1: "MAINT (PM)", 2: "REPLACEMENT (CM)"}

    print(" DDQN Training Start ")
    print("Environment: {num_components} Component. Max Age: {env.MAX_AGE_TERMINATION} hrs.")
    print("Gradient Clipping: Enabled (Norm=1.0). He Init: Enabled.")
    print("State: NORMALIZED [0,1]")

    for episode in range(1, EPISODES + 1):
        state, _ = env.reset()
        done = False
        episode_reward_raw = 0
        step_count = 0

        while not done:
            actions = agent.get_action(state)
            next_state, reward_raw, terminated, truncated, info = env.step(actions)
            done = terminated or truncated
            reward_scaled = reward_raw / REWARD_SCALE_FACTOR
            agent.memory.push(state, actions, reward_scaled, next_state, done)
            agent.train()

            # --- LOGGING: UN-NORMALIZE FOR DISPLAY ---
            comp_index = 0

            # Unpack Normalized State
            norm_abs_age, norm_weibull_age, norm_tsm, hazard, R, norm_rul, norm_mttf, L, norm_mttr, norm_aprev, _ = state[comp_index]

            # Un-Normalize for Human Readability
            MAX_TIME = env.MAX_AGE_TERMINATION
            disp_abs_age = norm_abs_age * MAX_TIME
            disp_weibull_age = norm_weibull_age * MAX_TIME
            disp_rul = norm_rul * MAX_TIME
            disp_mttr = norm_mttr * 2000.0 # Approx max MTTR scaling factor

            # Re-calculate Cost for display (env stores current base cost)
            current_base_op_cost = env.current_base_op_cost[comp_index]
            dynamic_op_cost = current_base_op_cost * (1.0 + env.OP_HAZARD_SCALE * hazard) # hazard is raw (but clipped at 1.0 in state, re-using here)

            print("\n[Ep: {episode} | Step: {step_count} | ABS TIME: {disp_abs_age:.0f} hrs | Epsilon: {agent.epsilon:.4f}]")
            print("  Component 1 ({env.component_names[comp_index]}):")
            print("    Weibull Age: {disp_weibull_age:.0f} hrs | Reliability: {R:.4f}")
            print("    Hazard Rate: {hazard:.6f} | RUL: {disp_rul:.0f} hrs")
            print("    Op Cost: {dynamic_op_cost:.2f} Rs/hr")
            print("    Action: {ACTION_MAP[actions[comp_index]]}")
            print("    Reward: {reward_raw:.0f}")

            state = next_state
            episode_reward_raw += reward_raw
            step_count += 1

        if episode % 10 == 0 or episode == EPISODES:
            print(f"\n--- Episode {episode}/{EPISODES} Summary ---")
            print("TOTAL COST (Raw): {episode_reward_raw:,.0f}")
            print("-" * 40)

    env.close()
    print("Training complete.")

if __name__ == "__main__":
    tf.config.experimental.set_visible_devices([], 'GPU')
    print("TensorFlow Version: {tf.__version__}")
    train_ddqn()

TensorFlow Version: 2.19.0
--- DDQN Training Start ---
Environment: 1 Component. Max Age: 10000.0 hrs.
Gradient Clipping: Enabled (Norm=1.0). He Init: Enabled.
State: NORMALIZED [0,1]

[Ep: 1 | Step: 0 | ABS TIME: 0 hrs | Epsilon: 1.0000]
  Component 1 (Electrodes):
    Weibull Age: 0 hrs | Reliability: 1.0000
    Hazard Rate: 0.000000 | RUL: 4009 hrs
    Op Cost: 2.00 Rs/hr
    Action: NOOP (Do Nothing)
    Reward: -2

[Ep: 1 | Step: 1 | ABS TIME: 1 hrs | Epsilon: 1.0000]
  Component 1 (Electrodes):
    Weibull Age: 1 hrs | Reliability: 1.0000
    Hazard Rate: 0.000006 | RUL: 4008 hrs
    Op Cost: 2.47 Rs/hr
    Action: NOOP (Do Nothing)
    Reward: -2

[Ep: 1 | Step: 2 | ABS TIME: 2 hrs | Epsilon: 1.0000]
  Component 1 (Electrodes):
    Weibull Age: 2 hrs | Reliability: 1.0000
    Hazard Rate: 0.000009 | RUL: 4007 hrs
    Op Cost: 2.72 Rs/hr
    Action: REPLACEMENT (CM)
    Reward: -30004

[Ep: 1 | Step: 3 | ABS TIME: 3 hrs | Epsilon: 1.0000]
  Component 1 (Electrodes):
    Weibull A

/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/input_layer.py:27: UserWarning: Argument `input_shape` is deprecated. Use `shape` instead.
  warnings.warn(


Streaming output truncated to the last 5000 lines.

[Ep: 1 | Step: 3564 | ABS TIME: 3564 hrs | Epsilon: 0.1708]
  Component 1 (Electrodes):
    Weibull Age: 593 hrs | Reliability: 0.8992
    Hazard Rate: 0.000288 | RUL: 3416 hrs
    Op Cost: 25.06 Rs/hr
    Action: REPLACEMENT (CM)
    Reward: -37583

[Ep: 1 | Step: 3565 | ABS TIME: 3565 hrs | Epsilon: 0.1707]
  Component 1 (Electrodes):
    Weibull Age: 0 hrs | Reliability: 1.0000
    Hazard Rate: 0.000000 | RUL: 4009 hrs
    Op Cost: 2.00 Rs/hr
    Action: NOOP (Do Nothing)
    Reward: -2

[Ep: 1 | Step: 3566 | ABS TIME: 3566 hrs | Epsilon: 0.1706]
  Component 1 (Electrodes):
    Weibull Age: 1 hrs | Reliability: 1.0000
    Hazard Rate: 0.000006 | RUL: 4008 hrs
    Op Cost: 2.47 Rs/hr
    Action: NOOP (Do Nothing)
    Reward: -2

[Ep: 1 | Step: 3567 | ABS TIME: 3567 hrs | Epsilon: 0.1705]
  Component 1 (Electrodes):
    Weibull Age: 2 hrs | Reliability: 1.0000
    Hazard Rate: 0.000009 | RUL: 4007 hrs
    Op Cost: 2.72 Rs/hr
    Acti